In [3]:
import os
import json
import pandas as pd
import traceback

In [19]:
import os
from dotenv import load_dotenv
from langchain_huggingface import HuggingFacePipeline


try:
    llm = HuggingFacePipeline.from_model_id(
        model_id="gpt2",  # نموذج مدعوم
        task="text-generation",

        model_kwargs={"temperature": 0.7, "do_sample": True}
    )
    output = llm("Hello world!")
    print("Output:", output)
except Exception as e:
    print("Error:", e)


KeyboardInterrupt: 

In [7]:
# Load model directly

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2")
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")
# Use a pipeline as a high-level helper

pipe = pipeline("text-generation", model="openai-community/gpt2")
llm = HuggingFacePipeline(pipeline=pipe)
print("Output:", output)



KeyboardInterrupt: 

In [4]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [132]:
key=os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [133]:
os.environ["HUGGINGFACEHUB_API_TOKEN"]=key

In [134]:
from langchain_huggingface import HuggingFaceEndpoint
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import HuggingFaceHub
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [234]:
llms = HuggingFaceEndpoint(
    huggingfacehub_api_token=key,
    repo_id="meta-llama/Llama-3.1-8B-Instruct",
    task="text-generation",
    temperature=0.7,# تقليل العشوائية
    do_sample= True,
    
)

In [235]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [236]:
TEMPLATE2="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [237]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [238]:
quiz_chain=LLMChain(llm=llms, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [239]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""


In [240]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)


In [241]:
review_chain=LLMChain(llm=llms, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)


In [242]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [243]:
file_path=r"D:\Ai generative ai\42\data.txt"


In [244]:
file_path


'D:\\Ai generative ai\\42\\data.txt'

In [245]:
with open(file_path, 'r') as file:
    TEXT = file.read()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 1912: character maps to <undefined>

In [246]:
NUMBER=12 
SUBJECT="biology"
TONE="very very Difficult"

In [247]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [248]:
response = generate_evaluate_chain.invoke({
    "text": TEXT,
    "number": NUMBER,
    "subject": SUBJECT,
    "tone": TONE,
    "response_json": json.dumps(RESPONSE_JSON)
})
print(response)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined

In [250]:
response["quiz"]

'5. Very difficult tone\n\n{\n"1": {"mcq": "What is the estimated age of emergence of life on Earth?", \n       "options": {"a": "More than 3.7 million years ago", \n                  "b": "Less than 3.7 million years ago",\n                  "c": "More than 3.7 billion years ago",\n                  "d": "Exactly 3.7 billion years ago"},\n       "correct": "c"},\n"2": {"mcq": "Which of the following types of organisms are not eukaryotic?",\n        "options": {"a": "Archaea", "b": "Bacteria", "c": "Protists", "d": "All of the above"},\n        "correct": "d"},\n"3": {"mcq": "According to the text, what is a key feature that ties biology together as a single field?",\n        "options": {"a": "Energy processing", "b": "Evolution", "c": "Hereditary information", "d": "All of the above"},\n        "correct": "d"},\n"4": {"mcq": "What is the term used to describe the ability of organisms to regulate their internal environments?",\n        "options": {"a": "Homeostasis", "b": "Heterosis", 

In [84]:
mcqs = json.loads(response['response_json'])

# طباعة الأسئلة بشكل مرتب
for qnum, qdata in mcqs.items():
    print(f"Question {qnum}: {qdata['mcq']}")
    for opt_key, opt_val in qdata['options'].items():
        print(f"  {opt_key.upper()}. {opt_val}")
    print(f"Correct answer: {qdata['correct'].upper()}\n")

Question 1: multiple choice question
  A. choice here
  B. choice here
  C. choice here
  D. choice here
Correct answer: CORRECT ANSWER

Question 2: multiple choice question
  A. choice here
  B. choice here
  C. choice here
  D. choice here
Correct answer: CORRECT ANSWER

Question 3: multiple choice question
  A. choice here
  B. choice here
  C. choice here
  D. choice here
Correct answer: CORRECT ANSWER



In [77]:
import re
import json

def extract_json_from_text(text):
    # يبحث عن أول { وآخر } ويقتطع النص بينهم
    match = re.search(r'\{.*\}', text, re.DOTALL)
    if not match:
        return None
    
    json_str = match.group(0)

    # محاولة إصلاح نقص الأقواس إذا موجود
    while json_str.count('{') > json_str.count('}'):
        json_str += '}'
    
    # محاولة تحميل JSON
    try:
        data = json.loads(json_str)
        return data
    except json.JSONDecodeError as e:
        print("خطأ في JSON:", e)
        return None


In [78]:
quiz_data = extract_json_from_text(quiz)
if quiz_data is None:
    print("لم أستطع استخراج JSON صالح من النص")
    return []


TypeError: expected string or bytes-like object, got 'DataFrame'

In [72]:
import json

# النص الكامل غير منسق بشكل جيد، لذا دعنا نفصل أول JSON فقط
raw_quiz = quiz


# نحاول استخلاص الجزء الذي يحتوي على JSON
import re

# استخراج أول JSON باستخدام regex
match = re.search(r'\{.*\}', raw_quiz, re.DOTALL)
if match:
    quiz_json = match.group(0)
    quiz_data = json.loads(quiz_json)
    print(quiz_data)
else:
    print("لم يتم العثور على JSON صالح.")


JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 329 (char 328)

In [73]:
print(quiz_data.items())


dict_items([('1', {'mcq': 'What is the scientific study of life known as?', 'options': {'a': 'Chemistry', 'b': 'Physics', 'c': 'Biology', 'd': 'Mathematics'}, 'correct': 'c'}), ('2', {'mcq': 'What is a major theme that explains the unity and diversity of life?', 'options': {'a': 'Evolution', 'b': 'Energy processing', 'c': 'Regulation of internal environment', 'd': 'All of the above'}, 'correct': 'd'}), ('3', {'mcq': 'What is the time period when life on Earth emerged?', 'options': {'a': 'Less than 1 billion years ago', 'b': 'Between 1-3.7 billion years ago', 'c': 'More than 3.7 billion years ago', 'd': 'More than 10 billion years ago'}, 'correct': 'c'}), ('4', {'mcq': 'What is the term used to describe the various forms of life on Earth?', 'options': {'a': 'Ecosystem', 'b': 'Biodiversity', 'c': 'Nutrient cycling', 'd': 'Biophysical environment'}, 'correct': 'b'}), ('5', {'mcq': 'What is the process by which organisms are able to move, grow, and reproduce?', 'options': {'a': 'Photosynth

In [74]:
quiz_table_data = []
for key, value in quiz_data.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [75]:
quiz_table_data


[{'MCQ': 'What is the scientific study of life known as?',
  'Choices': 'a: Chemistry | b: Physics | c: Biology | d: Mathematics',
  'Correct': 'c'},
 {'MCQ': 'What is a major theme that explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Energy processing | c: Regulation of internal environment | d: All of the above',
  'Correct': 'd'},
 {'MCQ': 'What is the time period when life on Earth emerged?',
  'Choices': 'a: Less than 1 billion years ago | b: Between 1-3.7 billion years ago | c: More than 3.7 billion years ago | d: More than 10 billion years ago',
  'Correct': 'c'},
 {'MCQ': 'What is the term used to describe the various forms of life on Earth?',
  'Choices': 'a: Ecosystem | b: Biodiversity | c: Nutrient cycling | d: Biophysical environment',
  'Correct': 'b'},
 {'MCQ': 'What is the process by which organisms are able to move, grow, and reproduce?',
  'Choices': 'a: Photosynthesis | b: Respiration | c: Energy processing | d: Metabolism',
  'Correct': 'c

In [76]:
quiz=pd.DataFrame(quiz_table_data)
quiz.to_csv("machinelearning.csv",index=False)


PermissionError: [Errno 13] Permission denied: 'machinelearning.csv'

In [33]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_24_2025_21_01_32'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2", use_auth_token=key)
model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2", use_auth_token=key)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
llms = HuggingFacePipeline(pipeline=pipe)
